In [13]:
import torch
from matplotlib import pyplot as plt
import cv2
from PIL import Image
import numpy as np
import pandas as pd
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from time import time

In [14]:
def gstreamer_pipeline(
    capture_width=1280,
    capture_height=720,
    display_width=1280,
    display_height=720,
    framerate=60,
    flip_method=0,
):
    return (
        "nvarguscamerasrc ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

In [25]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='exp16/weights/best.pt', force_reload=True)
model.classes = [0]

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/hong/.cache/torch/hub/master.zip
YOLOv5 🚀 v6.1-272-g8983324 Python-3.6.9 torch-1.10.2 CPU



Python 3.7.0 required by YOLOv5, but Python 3.6.9 is currently installed


Fusing layers... 
Model summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [26]:
def show_camera():
    # To flip the image, modify the flip_method parameter (0 and 2 are the most common)
    print(gstreamer_pipeline(flip_method=0))
    cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)
    if cap.isOpened():
        window_handle = cv2.namedWindow("CSI Camera", cv2.WINDOW_AUTOSIZE)
        # Window
        while cv2.getWindowProperty("CSI Camera", 0) >= 0:
            ret_val, img = cap.read()
            cv2.imshow("CSI Camera", img)
            # This also acts as
            keyCode = cv2.waitKey(30) & 0xFF
            # Stop the program on the ESC key
            if keyCode == 27:
                break
                
            elif keyCode == 32:
        # SPACE pressed
                img_name = "/home/hong/parking/parkinglot.png"
                cv2.imshow(img_name, img)
                cv2.imwrite(img_name, img)
                print("{} written!".format(img_name))
        cap.release()
        cv2.destroyAllWindows()
    else:
        print("Unable to open camera")


if __name__ == "__main__":
    show_camera()

nvarguscamerasrc ! video/x-raw(memory:NVMM), width=(int)1280, height=(int)720, format=(string)NV12, framerate=(fraction)60/1 ! nvvidconv flip-method=0 ! video/x-raw, width=(int)1280, height=(int)720, format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink
/home/hong/parking/parkinglot.png written!


In [20]:
img_name = "/home/hong/parking/parkinglot.png"

In [27]:
# Images
#dir = 'https://github.com/ultralytics/yolov5/raw/master/data/images/'
width = 1280
height = 720
dim = (width, height)
img = cv2.imread(img_name, cv2.IMREAD_UNCHANGED)
resized = cv2.resize(img, dim)


# Inference
results = model(resized)

# Results
results.print()  
results.save(labels=False)  # or .show()
#results.show()

# Data
#print(results.xyxy[0])  # print img1 predictions (pixels)
#                   x1           y1           x2           y2   confidence        class
# tensor([[7.50637e+02, 4.37279e+01, 1.15887e+03, 7.08682e+02, 8.18137e-01, 0.00000e+00],
#         [9.33597e+01, 2.07387e+02, 1.04737e+03, 7.10224e+02, 5.78011e-01, 0.00000e+00],
#         [4.24503e+02, 4.29092e+02, 5.16300e+02, 7.16425e+02, 5.68713e-01, 2.70000e+01]])

Saved 1 image to runs/detect/exp


image 1/1: 720x1280 (no detections)
Speed: 44.2ms pre-process, 734.7ms inference, 138.6ms NMS per image at shape (1, 3, 384, 640)


In [28]:
temp=results.pandas().xyxy[0]
temp1= pd.DataFrame(temp)

In [29]:
numpy_array = temp1.to_numpy()
np.savetxt("coordinates.txt", numpy_array, fmt = "%s", delimiter=' ')

In [30]:
filename = 'coordinates.txt'
data = np.genfromtxt(filename, delimiter=' ', dtype=None, names=('x1', 'y1', 'x2', 'y2', 'conf', 'class','name'))
print(data)

[]


/home/hong/parking/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: genfromtxt: Empty input file: "coordinates.txt"
  


In [31]:
model1 = tf.keras.models.load_model("/home/hong/parking/Model/saved_model.h5")
HEIGHT = 49
WIDTH = 37

In [41]:
def show_camera1():
    previous = time()
    delta = 0
    # To flip the image, modify the flip_method parameter (0 and 2 are the most common)
    print(gstreamer_pipeline(flip_method=0))
    cap1 = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)
    if cap1.isOpened():
        window_handle = cv2.namedWindow("CSI Camera", cv2.WINDOW_AUTOSIZE)
        # Window
        while cv2.getWindowProperty("CSI Camera", 0) >= 0:
            ret_val, img = cap1.read()
            cv2.imshow("CSI Camera", img)
            
            current = time()
            delta += current - previous
            previous = current
            
            # This also acts as
            keyCode = cv2.waitKey(30) & 0xFF
            # Stop the program on the ESC key
            if keyCode == 27:
                break
                
            elif delta > 30:
                img_name = "/home/hong/parking/test.png"
                cv2.imshow(img_name, img)
                cv2.imwrite(img_name, img)
                
                info1 = []

                with open(filename) as input_file:
    
                    for line in input_file:
                        x1, y1, x2, y2, conf, labels, name = (
                            item.strip() for item in line.rsplit(' ',6))
                        info1.append (dict(zip(('labels', 'conf', 'x1', 'y1', 'x2', 'y2'),
                                    (labels, conf, x1, y1, x2, y2))))
                
                for one_info in info1:
                    x1 = int(float(one_info["x1"]))
                    y1 = int(float(one_info["y1"]))
                    x2 = int(float(one_info["x2"]))
                    y2 = int(float(one_info["y2"]))
                    cropped_image = img1 [y1:y2, x1:x2]
                    image = cv2.resize(cropped_image, (WIDTH, HEIGHT))
                    image_x = np.expand_dims(image, axis=0)
                    image_x = tf.keras.applications.vgg16.preprocess_input(image_x)
                    prediction = model1.predict(image_x)
                    prediction = np.squeeze(prediction)

                    if prediction> 0.8:
                        cv2.rectangle(img1, (x1, y1), (x2, y2),(255, 0, 0), 3)
                        occupied = occupied + 1
        
                    else:
                        cv2.rectangle(img1, (x1, y1), (x2, y2),(0, 255, 0), 3)
                        empty = empty + 1
      
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(img1, str(occupied),
                        (0, 30), font,
                        1, (0, 255, 255),
                        4, cv2.LINE_AA)
                cv2.putText(img1, str(empty),
                        (0, 60), font,
                        1, (0, 255, 255),
                        4, cv2.LINE_AA)
                cv2.imshow("view", img1)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                cv2.waitKey(1)
                delta = 0
                cap1.release()
                cv2.destroyAllWindows()
                
        else:
            print("Unable to open camera")


if __name__ == "__main__":
    show_camera1()

IndentationError: expected an indented block (<ipython-input-41-988023b54aae>, line 33)

In [34]:
# Initialise variables to store current time difference as well as previous time call value
previous = time()
delta = 0
cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)
while True:
    # Get the current time, increase delta and update the previous variable
    current = time()
    delta += current - previous
    previous = current
    
    occupied = int(0)
    empty = int(0)
    
    if cap.isOpened():
        window_handle = cv2.namedWindow("CSI Camera", cv2.WINDOW_AUTOSIZE)
        # Window
        while cv2.getWindowProperty("CSI Camera", 0) >= 0:
            ret_val, img = cap.read()
            cv2.imshow("CSI Camera", img)
            
        if delta > 30:
            take_pic = cap.read()
            cv2.imwrite('/home/hong/parking/test.png', take_pic)
            img_name1 = cv2.imread('/home/hong/parking/test.png' , cv2.IMREAD_UNCHANGED)
            img1 = cv2.resize(img_name1, dim)
        
        if keyCode == 27:
                break

        info1 = []

        with open(filename) as input_file:
    
            for line in input_file:
                x1, y1, x2, y2, conf, labels, name = (
                    item.strip() for item in line.rsplit(' ',6))
                info1.append (dict(zip(('labels', 'conf', 'x1', 'y1', 'x2', 'y2'),
                                (labels, conf, x1, y1, x2, y2))))
                
        for one_info in info1:
            x1 = int(float(one_info["x1"]))
            y1 = int(float(one_info["y1"]))
            x2 = int(float(one_info["x2"]))
            y2 = int(float(one_info["y2"]))
            cropped_image = img1 [y1:y2, x1:x2]
            image = cv2.resize(cropped_image, (WIDTH, HEIGHT))
            image_x = np.expand_dims(image, axis=0)
            image_x = tf.keras.applications.vgg16.preprocess_input(image_x)
            prediction = model1.predict(image_x)
            prediction = np.squeeze(prediction)

            if prediction> 0.8:
                cv2.rectangle(img1, (x1, y1), (x2, y2),(255, 0, 0), 3)
                occupied = occupied + 1
        
            else:
                cv2.rectangle(img1, (x1, y1), (x2, y2),(0, 255, 0), 3)
                empty = empty + 1
      
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img1, str(occupied),
                (0, 30), font,
                1, (0, 255, 255),
                4, cv2.LINE_AA)
        cv2.putText(img1, str(empty),
                (0, 60), font,
                1, (0, 255, 255),
                4, cv2.LINE_AA)
        cv2.imshow("view", img1)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        delta = 0

error: OpenCV(4.1.1) /home/nvidia/host/build_opencv/nv_opencv/modules/highgui/src/window.cpp:352: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
